In [1]:
import pandas as pd
import json
import re
import src.cleaning as cl

In [212]:
df = pd.read_csv("./data/shots_v1.csv")

In [246]:
df.head()

,name,jornada,shot,coord_x,coord_y
0,J. Zyskowski,1,#local-in,24.750000,126.416664
1,J. Zyskowski,1,#local-in,24.750000,23.883333
2,J. Zyskowski,1,#local-out,25.705345,23.883333
3,J. Zyskowski,1,#local-in,27.616035,137.866669
4,J. Zyskowski,1,#local-in,27.128998,129.816666


In [214]:
df.rename(columns = {"0": "Jornada"}, inplace = True)
df.drop(columns = ["Unnamed: 0"], inplace = True)

In [215]:
names = cl.shots_per_player(df)

In [217]:
for elem in names:
    if elem == " ":
        names.remove(elem)

In [218]:
len(names)

37285

In [219]:
jornadas = cl.cumulat_shots(df)

In [221]:
len(jornadas)

33

In [222]:
# As we had problems trying to append the first gameday number of shots, we append it before looping. 
shots_per_jornada = [1157]
for i in range(len(jornadas)-1):
    shots_per_jornada.append(jornadas[i+1]-jornadas[i])

In [223]:
shots_per_jornada

[1157,
 1133,
 1141,
 1133,
 1133,
 1125,
 1135,
 1109,
 1112,
 1101,
 1146,
 1110,
 1107,
 1163,
 1173,
 1138,
 1051,
 1116,
 1140,
 1145,
 1158,
 985,
 1149,
 1157,
 1132,
 1159,
 1128,
 1141,
 1138,
 1147,
 1136,
 1166,
 1121]

In [224]:
num_jornada = list(range(1, 35))
num_jornada = [str(x) for x in num_jornada]

In [250]:
final_jornada = sum([[s] * n for s, n in zip(num_jornada, shots_per_jornada)], [])

In [252]:
#Finally we get a list with values from 1 to 34 (gamedays played) according to when every shot was made. 
len(final_jornada)

37285

### Now we are gonna divide the list "prueba" in order to get data for each three elements. 

In [227]:
#Our variable "prueba" contains all the information about shots every three values. 
len(prueba)

111855

### Here we divide the information contained in "prueba" in three lists. One to know if the shot was made or not, as a local or as a visitor. Another for its coordinate x and the last one for its coordinate y.

In [228]:
made = []
for i in range(0, len(prueba), 3):
    made.append(prueba[i])
    
coord_x = []
for i in range(1, len(prueba), 3):
    coord_x.append(prueba[i])
    
coord_y = []
for i in range(2, len(prueba), 3):
    coord_y.append(prueba[i])
    
cositas = list(zip(names, final_jornada, made, coord_x, coord_y))
df = pd.DataFrame(cositas)
df.rename(columns = {0: "name", 1: "jornada", 2: "shot", 3: "coord_x", 4: "coord_y"}, inplace = True)

### Aquí ya tenemos el dataset mínimamente decente. Falta cambiar coordenadas x de visitors, poner nombres bien y agrupar bien

In [254]:
df.head()

,name,jornada,shot,coord_x,coord_y
0,J. Zyskowski,1,#local-in,24.750000,126.416664
1,J. Zyskowski,1,#local-in,24.750000,23.883333
2,J. Zyskowski,1,#local-out,25.705345,23.883333
3,J. Zyskowski,1,#local-in,27.616035,137.866669
4,J. Zyskowski,1,#local-in,27.128998,129.816666


In [235]:
df["coord_x"] = pd.to_numeric(df["coord_x"], downcast="float")
df["coord_y"] = pd.to_numeric(df["coord_y"], downcast="float")
df["jornada"] = pd.to_numeric(df["jornada"], downcast="integer")

In [236]:
df_visitor = df.loc[(df["shot"] == "#visitor-in") | (df["shot"] == "#visitor-out") | (df["shot"] == "#visitor-dunk")]
df_local = df.loc[(df["shot"] == "#local-in") | (df["shot"] == "#local-out") | (df["shot"] == "#local-dunk")]
df_visitor["coord_x"] = abs(df_visitor["coord_x"] - 513)
df = pd.concat([df_local, df_visitor])
df["name"] = df["name"].str.strip()

,name,jornada,shot,coord_x,coord_y
58,A. Doornekamp,1,#visitor-in,418.221130,35.733334
59,A. Doornekamp,1,#visitor-out,473.481323,134.466660
60,A. Doornekamp,1,#visitor-in,355.336945,203.533340
61,A. Doornekamp,1,#visitor-in,458.720306,259.466675
62,A. Doornekamp,1,#visitor-out,432.513855,249.300003
...,...,...,...,...,...
37280,T. Abromaitis,33,#visitor-out,454.243286,241.250000
37281,T. Abromaitis,33,#visitor-in,465.201660,165.216660
37282,T. Abromaitis,33,#visitor-in,471.683014,259.200012
37283,T. Abromaitis,33,#visitor-in,466.288116,158.116669


In [245]:
df.to_csv("./data/database_shots.csv")

### Here we import our csv with every shot.

In [1175]:
super_llego_papa = pd.read_csv("./data/df_tiros_polygon.csv")

In [1177]:
super_llego_papa.sort_values('PLAYER_NAME', ascending = True, inplace = True)
super_llego_papa.drop(columns = ["Unnamed: 0", "index_right", "geometry", "coord_x", "coord_y"], inplace = True)

In [1178]:
super_llego_papa.shot = super_llego_papa.shot.apply(lambda x: 'in' if 'in' in x else x)
super_llego_papa.shot = super_llego_papa.shot.apply(lambda x: 'in' if 'dunk' in x else x)
super_llego_papa.shot = super_llego_papa.shot.apply(lambda x: 'out' if 'out' in x else x)

In [1179]:
super_llego_papa["PLAYER_NAME"] = super_llego_papa["PLAYER_NAME"].str.replace('á', 'a').str.replace('à', 'a').str.replace('é', 'e').str.replace('í', 'i').str.replace('ó', 'o').str.replace('Á', 'A').str.replace('Í', 'I').str.replace('ñ', 'n')
super_llego_papa["PLAYER_NAME"] = super_llego_papa["PLAYER_NAME"].str.replace('AJ Slaughter', 'A. Slaughter').str.replace('V. Gerun', 'V. Herun').str.replace('C. Pacheco', 'C. D. S. Pacheco').str.replace('F. Dos Anjos', 'F. D. Anjos').str.replace('P. Dime', 'M. Dime').str.replace('J. Gonzalez', 'C. Gonzalez').str.replace('W. Tavares', 'E. Tavares').str.replace('X. X. Lopez-Arostegui', 'X. Lopez-Arostegui').str.replace('A. Balcerowski', 'O. Balcerowski').str.replace('I. Llovet', 'N. Llovet')

<ipython-input-1179-aa87bd13e0d1>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  super_llego_papa["PLAYER_NAME"] = super_llego_papa["PLAYER_NAME"].str.replace('AJ Slaughter', 'A. Slaughter').str.replace('V. Gerun', 'V. Herun').str.replace('C. Pacheco', 'C. D. S. Pacheco').str.replace('F. Dos Anjos', 'F. D. Anjos').str.replace('P. Dime', 'M. Dime').str.replace('J. Gonzalez', 'C. Gonzalez').str.replace('W. Tavares', 'E. Tavares').str.replace('X. X. Lopez-Arostegui', 'X. Lopez-Arostegui').str.replace('A. Balcerowski', 'O. Balcerowski').str.replace('I. Llovet', 'N. Llovet')


In [1180]:
cositas = super_llego_papa.groupby(["name_left", "PLAYER_NAME","shot"]).count()
cositas.reset_index(inplace = True)

### The variable made represents all the shots which were in during the game.

In [1181]:
made = cositas.loc[cositas["shot"] == "in"]
made.drop(columns = ["shot"], inplace = True)
made.rename(columns = {"jornada": "shots_made"}, inplace = True)

In [1185]:
made1 = made.loc[made["name_left"] == "polygon1"]
made2 = made.loc[made["name_left"] == "polygon2"]
made3 = made.loc[made["name_left"] == "polygon3"]
made4 = made.loc[made["name_left"] == "polygon4"]
made5 = made.loc[made["name_left"] == "polygon5"]
made6 = made.loc[made["name_left"] == "polygon6"]
made7 = made.loc[made["name_left"] == "polygon7"]
made8 = made.loc[made["name_left"] == "polygon8"]
made9 = made.loc[made["name_left"] == "polygon9"]
made10 = made.loc[made["name_left"] == "polygon10"]
made11 = made.loc[made["name_left"] == "polygon11"]
made12 = made.loc[made["name_left"] == "polygon12"]

In [1187]:
#Aquí sacamos los totales
total = super_llego_papa.groupby(["name_left", "PLAYER_NAME"]).count()
total.reset_index(inplace = True)
total.drop(columns = ["shot"], inplace = True)
total.rename(columns = {"jornada": "shots_attempted"}, inplace = True)

In [1191]:
total1 = total.loc[total["name_left"] == "polygon1"]
total2 = total.loc[total["name_left"] == "polygon2"]
total3 = total.loc[total["name_left"] == "polygon3"]
total4 = total.loc[total["name_left"] == "polygon4"]
total5 = total.loc[total["name_left"] == "polygon5"]
total6 = total.loc[total["name_left"] == "polygon6"]
total7 = total.loc[total["name_left"] == "polygon7"]
total8 = total.loc[total["name_left"] == "polygon8"]
total9 = total.loc[total["name_left"] == "polygon9"]
total10 = total.loc[total["name_left"] == "polygon10"]
total11 = total.loc[total["name_left"] == "polygon11"]
total12 = total.loc[total["name_left"] == "polygon12"]

### Time to merge both DataFrames

In [1193]:
zona_1 = total1.merge(made1,how='left', left_on='PLAYER_NAME', right_on='PLAYER_NAME')
zona_1.fillna(0, inplace = True)
zona_1.drop(columns = ["name_left_x", "name_left_y"], inplace = True)
zona_1.rename(columns = {"shots_attempted": "zona1_attempted", "shots_made": "zona1_made", "PLAYER_NAME": "mergeo"}, inplace = True)

In [1194]:
zona_2 = total2.merge(made2,how='left', left_on='PLAYER_NAME', right_on='PLAYER_NAME')
zona_2.fillna(0, inplace = True)
zona_2.drop(columns = ["name_left_x", "name_left_y"], inplace = True)
zona_2.rename(columns = {"shots_attempted": "zona2_attempted", "shots_made": "zona2_made", "PLAYER_NAME": "mergeo"}, inplace = True)

In [1195]:
zona_3 = total3.merge(made3,how='left', left_on='PLAYER_NAME', right_on='PLAYER_NAME')
zona_3.fillna(0, inplace = True)
zona_3.drop(columns = ["name_left_x", "name_left_y"], inplace = True)
zona_3.rename(columns = {"shots_attempted": "zona3_attempted", "shots_made": "zona3_made", "PLAYER_NAME": "mergeo"}, inplace = True)

In [1196]:
zona_4 = total4.merge(made4,how='left', left_on='PLAYER_NAME', right_on='PLAYER_NAME')
zona_4.fillna(0, inplace = True)
zona_4.drop(columns = ["name_left_x", "name_left_y"], inplace = True)
zona_4.rename(columns = {"shots_attempted": "zona4_attempted", "shots_made": "zona4_made", "PLAYER_NAME": "mergeo"}, inplace = True)

In [1197]:
zona_5 = total5.merge(made5,how='left', left_on='PLAYER_NAME', right_on='PLAYER_NAME')
zona_5.fillna(0, inplace = True)
zona_5.drop(columns = ["name_left_x", "name_left_y"], inplace = True)
zona_5.rename(columns = {"shots_attempted": "zona5attempted", "shots_made": "zona5_made", "PLAYER_NAME": "mergeo"}, inplace = True)

In [1198]:
zona_6 = total6.merge(made6,how='left', left_on='PLAYER_NAME', right_on='PLAYER_NAME')
zona_6.fillna(0, inplace = True)
zona_6.drop(columns = ["name_left_x", "name_left_y"], inplace = True)
zona_6.rename(columns = {"shots_attempted": "zona6_attempted", "shots_made": "zona6_made", "PLAYER_NAME": "mergeo"}, inplace = True)

In [1199]:
zona_7 = total7.merge(made7,how='left', left_on='PLAYER_NAME', right_on='PLAYER_NAME')
zona_7.fillna(0, inplace = True)
zona_7.drop(columns = ["name_left_x", "name_left_y"], inplace = True)
zona_7.rename(columns = {"shots_attempted": "zona7_attempted", "shots_made": "zona7_made", "PLAYER_NAME": "mergeo"}, inplace = True)

In [1200]:
zona_8 = total8.merge(made8,how='left', left_on='PLAYER_NAME', right_on='PLAYER_NAME')
zona_8.fillna(0, inplace = True)
zona_8.drop(columns = ["name_left_x", "name_left_y"], inplace = True)
zona_8.rename(columns = {"shots_attempted": "zona8_attempted", "shots_made": "zona8_made", "PLAYER_NAME": "mergeo"}, inplace = True)

In [1201]:
zona_9 = total9.merge(made9,how='left', left_on='PLAYER_NAME', right_on='PLAYER_NAME')
zona_9.fillna(0, inplace = True)
zona_9.drop(columns = ["name_left_x", "name_left_y"], inplace = True)
zona_9.rename(columns = {"shots_attempted": "zona9_attempted", "shots_made": "zona9_made", "PLAYER_NAME": "mergeo"}, inplace = True)

In [1202]:
zona_10 = total10.merge(made10,how='left', left_on='PLAYER_NAME', right_on='PLAYER_NAME')
zona_10.fillna(0, inplace = True)
zona_10.drop(columns = ["name_left_x", "name_left_y"], inplace = True)
zona_10.rename(columns = {"shots_attempted": "zona10_attempted", "shots_made": "zona10_made", "PLAYER_NAME": "mergeo"}, inplace = True)

In [1203]:
zona_11 = total11.merge(made11,how='left', left_on='PLAYER_NAME', right_on='PLAYER_NAME')
zona_11.fillna(0, inplace = True)
zona_11.drop(columns = ["name_left_x", "name_left_y"], inplace = True)
zona_11.rename(columns = {"shots_attempted": "zona11_attempted", "shots_made": "zona11_made", "PLAYER_NAME": "mergeo"}, inplace = True)

In [1204]:
zona_12 = total12.merge(made12,how='left', left_on='PLAYER_NAME', right_on='PLAYER_NAME')
zona_12.fillna(0, inplace = True)
zona_12.drop(columns = ["name_left_x", "name_left_y"], inplace = True)
zona_12.rename(columns = {"shots_attempted": "zona12_attempted", "shots_made": "zona12_made", "PLAYER_NAME": "mergeo"}, inplace = True)

### Importamos DataFrame de estadísticas generales

In [34]:
average_stats = pd.read_csv("./data/acb_stats.csv")

In [35]:
average_stats.drop(columns = ["Unnamed: 0"], inplace = True)
average_stats.rename(columns = {"Player": "PLAYER_NAME"}, inplace = True)
average_stats.sort_values('PLAYER_NAME', ascending = True, inplace = True)
average_stats.head()

,#,PLAYER_NAME,Team,GP,MIN,FGM,FGA,FG%,3PM,3PA,...,DRB,REB,AST,STL,BLK,PTS,Pos,Height,Weight,Age
124,1,A.J. Slaughter,GCN,22,24.3,5.0,10.2,0.484,1.7,4.2,...,1.9,2.3,3.7,1.1,0.2,13.7,G,6-4,185,33
174,5,Aaron Doornekamp,CAN,31,25.6,2.6,6.1,0.431,2.0,4.6,...,2.9,3.5,1.3,0.8,0.1,8.4,PF,6-7,212,35
274,15,Aaron Jones,BBB,8,19.7,1.6,3.9,0.419,0.4,1.0,...,1.5,3.8,1.0,0.5,0.4,4.1,F,6-9,220,27
281,2,Achille Polonara,CAJ,33,26.5,4.2,8.8,0.479,1.4,4.1,...,3.8,5.5,1.9,1.1,0.5,11.3,PF,6-8,198,29
23,11,Adam Hanga,FCB,31,16.1,2.2,3.7,0.578,1.1,2.0,...,2.8,3.4,2.1,0.8,0.4,6.2,SF,6-7,200,31


In [1258]:
average_stats["PLAYER_NAME"] = average_stats["PLAYER_NAME"].str.replace('Osas Ehigitor', 'Osas Ehigiator').str.replace('T. Mcfadden', 'T. McFadden').str.replace('D. Dileo', 'D. DiLeo').str.replace('Tunde Olumuyiwa', 'B. Olumuyiwa').str.replace('K. C. Jr.', 'K. Cheatham').str.replace('T. Bray', 'TJ Bray').str.replace('J. Barea', 'JJ Barea').str.replace('S. V. Rossom', 'S. Van Rossom').str.replace('J. W. Iii', 'J. Webb').str.replace('M. Kingsley', 'K. Moses').str.replace('A. D. Valle', 'A. Della Valle').str.replace('D. Rendl', 'J. Randle').str.replace('R. S. Miguel', 'R. San Miguel').str.replace('T. Campbell', 'TJ Campbell').str.replace('F. S. Emeterio', 'F. San Emeterio')

<ipython-input-1258-5453df86f208>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  average_stats["PLAYER_NAME"] = average_stats["PLAYER_NAME"].str.replace('Osas Ehigitor', 'Osas Ehigiator').str.replace('T. Mcfadden', 'T. McFadden').str.replace('D. Dileo', 'D. DiLeo').str.replace('Tunde Olumuyiwa', 'B. Olumuyiwa').str.replace('K. C. Jr.', 'K. Cheatham').str.replace('T. Bray', 'TJ Bray').str.replace('J. Barea', 'JJ Barea').str.replace('S. V. Rossom', 'S. Van Rossom').str.replace('J. W. Iii', 'J. Webb').str.replace('M. Kingsley', 'K. Moses').str.replace('A. D. Valle', 'A. Della Valle').str.replace('D. Rendl', 'J. Randle').str.replace('R. S. Miguel', 'R. San Miguel').str.replace('T. Campbell', 'TJ Campbell').str.replace('F. S. Emeterio', 'F. San Emeterio')


In [1259]:
nombres = list(average_stats["PLAYER_NAME"])
cambio_nombre = []
for s in nombres:
    l = s.split()
    new = ""
    for i in range(len(l)-1):
        s = l[i]
        new += (s[0].upper()+'. ')
    new += l[-1].title()
    cambio_nombre.append(new)

In [1261]:
average_stats["mergeo"] = cambio_nombre
average_stats["mergeo"] = average_stats["mergeo"].str.replace('O. Ehigitor', 'O. Ehigiator').str.replace('T. Mcfadden', 'T. McFadden').str.replace('D. Dileo', 'D. DiLeo').str.replace('T. Olumuyiwa', 'B. Olumuyiwa').str.replace('K. C. Jr.', 'K. Cheatham').str.replace('T. Bray', 'TJ Bray').str.replace('J. Barea', 'JJ Barea').str.replace('S. V. Rossom', 'S. Van Rossom').str.replace('J. W. Iii', 'J. Webb').str.replace('M. Kingsley', 'K. Moses').str.replace('A. D. Valle', 'A. Della Valle').str.replace('D. Rendl', 'J. Randle').str.replace('R. S. Miguel', 'R. San Miguel').str.replace('T. Campbell', 'TJ Campbell').str.replace('F. S. Emeterio', 'F. San Emeterio')
average_stats.head()

In [1264]:
final = average_stats.merge(zona_1, how = 'left', left_on='mergeo', right_on='mergeo')
final = final.merge(zona_2, how = 'left', left_on='mergeo', right_on='mergeo')
final = final.merge(zona_3, how = 'left', left_on='mergeo', right_on='mergeo')
final = final.merge(zona_4, how = 'left', left_on='mergeo', right_on='mergeo')
final = final.merge(zona_5, how = 'left', left_on='mergeo', right_on='mergeo')
final = final.merge(zona_6, how = 'left', left_on='mergeo', right_on='mergeo')
final = final.merge(zona_7, how = 'left', left_on='mergeo', right_on='mergeo')
final = final.merge(zona_8, how = 'left', left_on='mergeo', right_on='mergeo')
final = final.merge(zona_9, how = 'left', left_on='mergeo', right_on='mergeo')
final = final.merge(zona_10, how = 'left', left_on='mergeo', right_on='mergeo')
final = final.merge(zona_11, how = 'left', left_on='mergeo', right_on='mergeo')
final = final.merge(zona_12, how = 'left', left_on='mergeo', right_on='mergeo')

In [1265]:
final.drop(columns = ["mergeo"], inplace = True)
final.fillna(0, inplace= True)
final = final.loc[final["GP"] > 7]
final.head()

In [1275]:
final.to_csv("./data/final_average_stats.csv")